## Interactuando con Github

In [1]:
from github import Github
import os
from dotenv import load_dotenv, find_dotenv
from time import sleep
import re
import unicodedata
import json
import pickle

load_dotenv(find_dotenv())


git_key = os.getenv("GITHUB_ADMIN_TOKEN")
g = Github(git_key)

def download_github_data():
    with open("files/github_books.pkl", "wb") as file:
        books = _get_books()
        pickle.dump(books, file)


def load_github_data():
    books = []
    with open("files/github_books.pkl", "rb") as file:
        try:
            while True:
                books.append(pickle.load(file))
        except EOFError:
            return books[0]
        
def _get_books():
    """Comprueba si un repositorio tiene archivo SUMMARY.md, indicativo de que
    se trata de un Gitbook.

    Devuelve:
        listado de gitbook de la organización CATEDU.
    """
    global g
    repos = list(g.get_organization("catedu").get_repos())
    gitbooks = [repo for repo in repos if _has_summary(repo)]
    return gitbooks

def _has_summary(repo):
    """Comprueba si un repositorio tiene archivo SUMMARY.md, indicativo de que
    se trata de un Gitbook.

    Parámetros:
        repo (Objeto PyGithub que representa un repositorio)

    Devuelve:
        True or False (Boolean) indicando si tiene o no SUMMARY.md
    """
    try:
        contents = [content.name for content in repo.get_contents("")]
        it_has_summary = "SUMMARY.md" in contents
        return it_has_summary
    except:
        print(f"######### El repo {repo.name} falla #############")
        
def repos_data():
    books = load_github_data()
    data = [
        {
            "repo_name": book.name,
            "master_url": book.html_url,
            "gh-pages": f"https://catedu.github.io/{book.name}/",
        }
        for book in books
    ]
    return data

In [11]:
download_github_data()

In [2]:
books = load_github_data()

In [14]:
books[30]

Repository(full_name="catedu/robotica-educativa-con-mbot-II")

## Importar libros de Github a Wagtail

Para esta tarea tenemos que:

1. Descargar el SUMMARY.md de cada libro
2. Por cada apartado guardar:
  * Orden
  * Título
  * Ruta del archivo.md
  * Quién es el padre
3. Cambiar de rama a gh-pages:
  * Obtener el html de cada archivo
  * Descargar imágenes.
    * Si es ruta absoluta, comprobar si viene del mismo repo y bajar
  * Guardar html, sustituyendo la ruta de los archivos locales.
4. Crear páginas en Wagtail dividiendo html y bloques de código

### Obteniendo el índice

In [15]:
def get_summary(book):
    summary = book.get_contents("SUMMARY.md")
    print(book.name)
    return summary

summary = get_summary(books[30])

robotica-educativa-con-mbot-II


In [17]:
summary.decoded_content.decode().split("\n")

['# Summary',
 '',
 '* [Introducción](README.md)',
 '',
 '## 1. Veamos ejemplos',
 '',
 '* [M1 mBot plus](index0.md)',
 '* [Repaso de mBot](repaso_de_mbot.md)',
 '* [Robot polilla](robot_polilla.md)',
 '* [Come-dinosaurio](comedinosaurio.md)',
 '* [Seguir circuito y ultrasonidos](seguir_circuito_y_ultrasonidos.md)',
 '* [Evitar caerse](evitar_caerse.md)',
 '* [Radar](radar.md)',
 '* [Matemáticas y robótica](matemticas_y_robtica.md)',
 '* [mBot en Infantil](mbot_en_infantil.md)',
 '* [Al infinito y más allá...](al_infinito_y_ms_all.md)',
 '',
 '## 2. Módulos y accesorios',
 '',
 '* [M2 mBot plus](index1.md)',
 '* [Acelerómetro y Gyro 3 ejes](acelermetro_y_gyro_3_ejes.md)',
 '* [Potenciómetro](potencimetro.md)',
 '* [Sensor sonido](sensor_sonido.md)',
 '* [Sensor IR](sensor_ir.md)',
 '* [Sensor PIR](sensor_pir.md)',
 '',
 '## 3. Escarabajos, brazos y garras',
 '',
 '* [M3 plus servos, garras y escarabajos](index2.md)',
 '* [Escarabajos](escarabajos.md)',
 '* [Ojo con los servos](ojo_con_

### Descargando imágenes

In [20]:
contents = books[0].get_contents("")

while contents:
    file_content = contents.pop(0)
    if file_content.type == "dir":
        contents.extend(books[0].get_contents(file_content.path))
    else:
        print(file_content.download_url)

https://raw.githubusercontent.com/catedu/abies-lectores/master/.gitignore
https://raw.githubusercontent.com/catedu/abies-lectores/master/.nojekyll
https://raw.githubusercontent.com/catedu/abies-lectores/master/FOOTER.md
https://raw.githubusercontent.com/catedu/abies-lectores/master/LICENSE
https://raw.githubusercontent.com/catedu/abies-lectores/master/README.md
https://raw.githubusercontent.com/catedu/abies-lectores/master/SUMMARY.md
https://raw.githubusercontent.com/catedu/abies-lectores/master/book.json
https://raw.githubusercontent.com/catedu/abies-lectores/master/consultar_catlogos_de_otras_bibliotecas.md
https://raw.githubusercontent.com/catedu/abies-lectores/master/consultar_el_catlogo_de_fondos_de_la_biblioteca_de_mi_centro.md
https://raw.githubusercontent.com/catedu/abies-lectores/master/crditos.md
https://raw.githubusercontent.com/catedu/abies-lectores/master/darme_de_alta_en_abiesweb_para_usar_la_biblioteca_de_mi_centro_escolar.md
https://raw.githubusercontent.com/catedu/abie

In [23]:
os.system("you-get -o files -O test https://raw.githubusercontent.com/catedu/abies-lectores/master/img/consultarOpiniones.JPG")

0